<a href="https://colab.research.google.com/github/rsvolkert/Beer-Recipes/blob/main/generate_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>